# Step 2: Dimensionality Reduction on Gene Expression Counts

## What is this step doing?

We have a dataset where each data point (a tissue "spot") is described by **33,538 features** — the raw integer count of how many mRNA molecules of each gene were detected at that location. Our goal downstream is to train a small neural network, so we need a compact, informative representation of each spot first.

This is a standard **unsupervised dimensionality reduction** problem:

$$\mathbf{X} \in \mathbb{N}^{N \times G} \xrightarrow{\text{dim. reduction}} \mathbf{A} \in \mathbb{R}^{N \times K}$$

with $N \approx 4{,}000$ spots, $G = 33{,}538$ genes, $K = 14$ latent dimensions.

The output matrix $\mathbf{A}$ becomes the **reconstruction target** for the GASTON autoencoder in the next step: the decoder must reproduce $\mathbf{A}_i$ from a single scalar coordinate per spot.

---

## Several methods could work here — this is an open design choice

This step is not unique to spatial transcriptomics. Any method that produces a fixed-size embedding from a high-dimensional feature vector is a candidate. Here are the main options:

| Method | Core idea | Notes for this task |
|--------|-----------|---------------------|
| **PCA** | SVD on the data matrix; find $K$ directions of max variance | Fast and simple. Works reasonably well after a log-transform of counts (`log(X+1)`). A perfectly valid baseline. |
| **NMF** | $\mathbf{X} \approx \mathbf{Z}\mathbf{W}^\top$ with $\mathbf{Z}, \mathbf{W} \geq 0$ | Non-negativity gives interpretable parts-based factors. Also works after log-transform. Good alternative to try. |
| **scVI** | VAE with a Zero-Inflated Negative Binomial (ZINB) decoder — learns a stochastic latent code | Most expressive model; handles dropout noise best. Slower to train. Latent space is stochastic (reparameterization trick), so using it as a regression target needs care. Worth trying as an ablation. |
| **GLM-PCA** *(used here)* | Matrix factorization fitted by Poisson MLE — replaces the Gaussian reconstruction loss with a count-appropriate likelihood | Designed specifically for raw count matrices; no pre-normalization needed. The GASTON paper uses this, so we follow it as our default. |

All four methods output an $(N, K)$ embedding matrix that can slot directly into the rest of the GASTON pipeline. **Swapping this step is one of the cleanest ablations you can run** — same neural network, different input representation.

---

## GLM-PCA — the model we use

GLM-PCA (Townes et al., *Genome Biology* 2019) replaces the Frobenius loss in PCA with a **Poisson log-likelihood**:

$$X_{ig} \sim \text{Poisson}\!\left(\exp\!\left(\mathbf{z}_i^\top \mathbf{w}_g + \mu_g\right)\right)$$

- $\mathbf{z}_i \in \mathbb{R}^K$ — latent embedding for spot $i$ (stacked into output matrix **A**)
- $\mathbf{w}_g \in \mathbb{R}^K$ — loading vector for gene $g$
- $\mu_g \in \mathbb{R}$ — gene-specific intercept (absorbs baseline expression level)

Fitting = **MLE**: maximize $\sum_{i,g} \log p(X_{ig} \mid \mathbf{z}_i, \mathbf{w}_g, \mu_g)$ jointly over all parameters.
Optimization uses alternating Newton updates — conceptually similar to coordinate descent.

The log-link $\exp(\cdot)$ ensures predicted counts are always non-negative, which is consistent with how count data actually behaves.

---

## Output of this notebook

```
glmpca.npy    shape (N, K=14)  — embedding A, reconstruction target for GASTON decoder
coords_mat.npy  shape (N, 2)   — (x, y) spatial coordinates, input to GASTON encoder
```

In [ ]:
# =============================================================================
# Step 2: GLM-PCA — Dimensionality Reduction on Raw Gene Expression Counts
# =============================================================================
# EECS 545 Project: C-GASTON
# Sections processed: 151507, 151508 (Sample 1) | 151673, 151674 (Sample 3)
#
# WHY THIS STEP EXISTS
# --------------------
# Our gene expression matrix X has shape (N, G) = (~4000 spots, 33538 genes).
# That's 33k-dimensional input — way too high for the GASTON neural network to
# train on directly.  We need a compact, informative embedding.
#
# Why not just use standard PCA?
# PCA assumes continuous, roughly Gaussian features.  Gene expression data is
# raw integer counts (UMIs) that follow a Poisson or Negative-Binomial
# distribution — extremely sparse (>90% zeros) and highly skewed.  Applying
# PCA to count data violates its assumptions and gives poor latent factors.
#
# GLM-PCA (Townes et al., Genome Biology 2019) fixes this by replacing the
# Gaussian likelihood in PCA with a Poisson likelihood.  It's essentially
# a matrix factorization with a log-linear model:
#
#   X_{ig} ~ Poisson( exp( z_i^T w_g + mu_g ) )
#
# where:
#   z_i  in R^K  = latent factor for spot i       (what we want)
#   w_g  in R^K  = loading vector for gene g
#   mu_g in R    = gene-specific intercept (library size offset)
#
# The parameters {z_i, w_g, mu_g} are found by maximum likelihood — the same
# MLE framework we covered in EECS 545.
#
# OUTPUT
# ------
# A = [z_1, z_2, ..., z_N]^T   shape: (N, K=14)
# This K=14 dimensional embedding is what GASTON's decoder tries to reconstruct
# from the scalar isodepth.  It acts as the "compressed ground truth" for
# the autoencoder reconstruction loss.

import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from glmpca import glmpca

plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150

print("Libraries imported.")

In [ ]:
# =============================================================================
# Paths
# =============================================================================

BASE_DIR = '/home/siruilf/A_new_dataset_for_gaston'

SAMPLE1_DATA_DIR = f'{BASE_DIR}/ST_Data/DLPFC_sample1/original_data'
SAMPLE3_DATA_DIR = f'{BASE_DIR}/ST_Data/DLPFC_sample3/original_data'

GLMPCA_OUTPUT_DIR = f'{BASE_DIR}/2.GLM_PC/glmpca_results'
os.makedirs(GLMPCA_OUTPUT_DIR, exist_ok=True)

print(f"GLM-PCA output directory: {GLMPCA_OUTPUT_DIR}")

In [ ]:
# =============================================================================
# Slices to process
# =============================================================================
# We use 2 sections from each of the two donors:
#   Sample 1 (donor 1): 151507, 151508
#   Sample 3 (donor 2): 151673, 151674
# Each section is processed independently — GLM-PCA is per-slice.

slices_to_process = {
    '151507': f'{SAMPLE1_DATA_DIR}/151507.h5ad',
    '151508': f'{SAMPLE1_DATA_DIR}/151508.h5ad',
    '151673': f'{SAMPLE3_DATA_DIR}/151673.h5ad',
    '151674': f'{SAMPLE3_DATA_DIR}/151674.h5ad',
}

print("Slices to process:")
for sid, path in slices_to_process.items():
    print(f"  {sid}: {path}")

In [ ]:
# =============================================================================
# GLM-PCA Hyperparameters
# =============================================================================
#
# K = num_dims: number of latent dimensions
# -----------------------------------------
# Following the GASTON paper and tutorial, we set K = 2 * Q, where Q is the
# number of spatial domains.  For DLPFC: Q=7 (layers L1-L6 + white matter),
# so K = 14.  The factor of 2 gives the decoder enough capacity to represent
# both the mean expression and its spatial gradient per domain.
# (Verified from GASTON cerebellum tutorial: num_dims=8 for Q=4 clusters.)
#
# penalty
# -------
# L2 regularization on the loadings W.  Prevents overfitting in the MLE
# optimization — same role as weight decay in gradient descent (EECS 545).
#
# num_iters / eps
# ---------------
# GLM-PCA is optimized by an alternating MLE procedure (similar to EM).
# num_iters caps the outer loop; eps is the convergence threshold on
# relative change in deviance (= -2 * log-likelihood).
#
# num_genes
# ---------
# We feed ALL 33,538 genes to GLM-PCA (num_genes=None).
# If you want faster runs, set num_genes=30000 to use only the top 30k genes
# ranked by total UMI count — this is the strategy from the GASTON tutorial.
# Note: GASTON does NOT use HVG (highly variable gene) selection here.
#       Genes are ranked by total expression, not variability.

num_dims  = 14      # K = 2 * Q = 2 * 7
penalty   = 10      # L2 regularization on loadings W
num_iters = 30      # max outer iterations
eps       = 1e-4    # convergence threshold on relative deviance change
num_genes = None    # None = use all genes; set 30000 for faster runs
spot_umi_threshold = 0  # drop spots with total UMI below this (0 = keep all)

print("GLM-PCA parameters:")
print(f"  K (latent dims)  : {num_dims}  [ = 2 x 7 DLPFC layers ]")
print(f"  penalty          : {penalty}")
print(f"  max iterations   : {num_iters}")
print(f"  convergence eps  : {eps}")
print(f"  genes used       : {'all' if num_genes is None else num_genes}")

In [ ]:
# =============================================================================
# Helper functions
# =============================================================================

def load_and_extract(h5ad_path, spot_umi_threshold=0):
    """
    Load one h5ad file and extract the three arrays GASTON needs.

    Returns
    -------
    counts_mat : ndarray (N, G)  raw UMI counts  — the design matrix X
    coords_mat : ndarray (N, 2)  (x, y) spatial coordinates
    gene_labels: ndarray (G,)   gene names
    """
    adata = sc.read_h5ad(h5ad_path)

    # .X may be a scipy sparse matrix; convert to dense numpy array
    counts_mat = adata.X.toarray() if hasattr(adata.X, 'toarray') else np.array(adata.X)
    coords_mat = adata.obsm['spatial']
    gene_labels = np.array(adata.var_names)

    # Optional: remove very low-quality spots (total UMI < threshold)
    if spot_umi_threshold > 0:
        keep = np.sum(counts_mat, axis=1) >= spot_umi_threshold
        print(f"  Dropping {(~keep).sum()} spots below UMI threshold {spot_umi_threshold}")
        counts_mat = counts_mat[keep]
        coords_mat = coords_mat[keep]

    return counts_mat, coords_mat, gene_labels


def run_glmpca(counts_mat, num_dims, penalty, num_iters, eps, num_genes=None):
    """
    Fit GLM-PCA on one slice's count matrix.

    The model optimized here is:
        X_{ig} ~ Poisson( exp( z_i^T w_g + mu_g ) )
    Parameters {z_i, w_g, mu_g} are estimated by MLE.
    Optimization is an alternating least-squares / Newton procedure
    that minimizes the Poisson deviance (= -2 * log-likelihood).

    The glmpca library expects shape (G, N) — genes as rows — so we transpose.

    Returns
    -------
    A : ndarray (N, K)   latent factor matrix  [ = z_i stacked as rows ]
    """
    # Gene selection: top genes by total UMI count (not HVG)
    if num_genes is None or num_genes >= counts_mat.shape[1]:
        X = counts_mat
        print(f"  Using all {X.shape[1]} genes")
    else:
        top_idx = np.argsort(np.sum(counts_mat, axis=0))[-num_genes:]
        X = counts_mat[:, top_idx]
        print(f"  Using top {X.shape[1]} genes by total UMI")

    # GLM-PCA call — matches GASTON tutorial exactly
    # Input shape: (G, N)  (library convention)
    res = glmpca.glmpca(
        X.T,
        num_dims,
        fam="poi",
        penalty=penalty,
        verbose=True,
        ctl={"maxIter": num_iters, "eps": eps, "optimizeTheta": True}
    )

    A = res['factors']   # shape (N, K)
    return A


print("Functions defined.")

In [ ]:
# =============================================================================
# Run GLM-PCA on each slice and save outputs
# =============================================================================
# Runtime note: each slice takes roughly 10–30 min depending on hardware.
# The deviance printed each iteration should decrease monotonically.
# Convergence is reached when relative change < eps.
#
# What we save
# ------------
# counts_mat.npy  : raw X   (N, G)   — needed if you want to re-run GLM-PCA
# coords_mat.npy  : (x,y)   (N, 2)   — spatial inputs to GASTON encoder
# gene_labels.npy : gene names (G,)  — bookkeeping
# glmpca.npy      : A       (N, K)   — reconstruction TARGET for GASTON decoder

glmpca_results = {}

for slice_id, h5ad_path in slices_to_process.items():
    print(f"\n{'='*60}")
    print(f"Slice: {slice_id}")
    print(f"{'='*60}")

    # --- Step 1: load raw data ---
    print("[1/3] Loading data...")
    counts_mat, coords_mat, gene_labels = load_and_extract(h5ad_path, spot_umi_threshold)
    print(f"      counts_mat : {counts_mat.shape}  (N spots x G genes)")
    print(f"      coords_mat : {coords_mat.shape}  (N spots x 2 coordinates)")

    # --- Step 2: fit GLM-PCA ---
    # This is the slow step.  GLM-PCA fits the Poisson log-linear model by MLE,
    # compressing 33k genes -> K=14 latent factors per spot.
    print("\n[2/3] Fitting GLM-PCA  (watch deviance decrease each iteration)...")
    A = run_glmpca(counts_mat, num_dims, penalty, num_iters, eps, num_genes)
    print(f"      Output A   : {A.shape}  (N spots x K={num_dims} factors)")

    # --- Step 3: save ---
    print("\n[3/3] Saving...")
    out_dir = f'{GLMPCA_OUTPUT_DIR}/{slice_id}'
    os.makedirs(out_dir, exist_ok=True)

    np.save(f'{out_dir}/counts_mat.npy',  counts_mat)
    np.save(f'{out_dir}/coords_mat.npy',  coords_mat)
    np.save(f'{out_dir}/gene_labels.npy', gene_labels)
    np.save(f'{out_dir}/glmpca.npy',      A)
    print(f"      Saved to: {out_dir}/")

    glmpca_results[slice_id] = {
        'counts_mat': counts_mat,
        'coords_mat': coords_mat,
        'gene_labels': gene_labels,
        'A': A,
    }

print(f"\n{'='*60}")
print("All slices done.")
print(f"{'='*60}")

In [ ]:
# =============================================================================
# Visualization: plot all K=14 latent factors spatially
# =============================================================================
# Each panel below shows one dimension z_k of the embedding, colored by value
# and plotted at the (x, y) coordinates of each spot.
#
# What to look for:
#   - Factors that show smooth spatial gradients aligned with cortical layers
#     are the informative ones — they capture the expression variation that
#     GASTON will learn to explain with a 1D isodepth coordinate.
#   - Noisy or salt-and-pepper factors carry less signal.
#
# This is analogous to visualizing the principal components after PCA:
# the first few components capture the dominant structure.

def plot_glmpca_factors(A, coords_mat, slice_id, save_path=None):
    """
    Plot all K latent factors as spatial scatter plots.
    Layout: 2 rows x (K/2) columns.
    """
    K = A.shape[1]
    ncols = K // 2
    fig, axes = plt.subplots(2, ncols, figsize=(4 * ncols, 8))

    for k in range(K):
        r, c = divmod(k, ncols)
        ax = axes[r, c]
        sc = ax.scatter(
            coords_mat[:, 0], coords_mat[:, 1],
            c=A[:, k], cmap='RdBu_r', s=3, rasterized=True
        )
        ax.set_title(f'PC {k+1}', fontsize=9)
        ax.set_aspect('equal')
        ax.axis('off')
        plt.colorbar(sc, ax=ax, shrink=0.6, pad=0.02)

    plt.suptitle(
        f'GLM-PCA latent factors — {slice_id}\n'
        f'(K={K} dims, model: X_ig ~ Poisson(exp(z_i^T w_g + mu_g)))',
        fontsize=11, fontweight='bold'
    )
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Saved: {save_path}")
    plt.show()
    plt.close()


for slice_id, data in glmpca_results.items():
    print(f"\nPlotting factors for {slice_id}...")
    save_path = f'{GLMPCA_OUTPUT_DIR}/{slice_id}/glmpca_factors.png'
    plot_glmpca_factors(data['A'], data['coords_mat'], slice_id, save_path)

In [ ]:
# =============================================================================
# Quick sanity check: variance explained per factor
# =============================================================================
# The factors from GLM-PCA are NOT orthogonal (unlike PCA), so there is no
# single "variance explained" metric.  But we can check how much variance
# each factor z_k has across spots — a factor with near-zero variance across
# spots is essentially unused.

print("Factor standard deviations across spots (higher = more active factor):\n")
print(f"{'Slice':<10}  " + "  ".join([f"PC{k+1:02d}" for k in range(num_dims)]))
print("-" * (10 + num_dims * 6))

for slice_id, data in glmpca_results.items():
    stds = data['A'].std(axis=0)
    std_str = "  ".join([f"{s:.2f}" for s in stds])
    print(f"{slice_id:<10}  {std_str}")

print()
print("Note: factors are not orthogonal — std is only a rough activity measure.")

In [ ]:
# =============================================================================
# Summary
# =============================================================================
# This step reduced the gene expression data from G=33,538 dimensions
# down to K=14 latent factors per spot, using a Poisson log-linear model
# fitted by MLE.
#
# Pipeline role
# -------------
#   Raw counts X (N x 33538)
#       |  GLM-PCA (this notebook)
#       v
#   Latent factors A (N x 14)    <-- GASTON decoder reconstruction target
#   Spatial coords S (N x 2)     <-- GASTON encoder input
#
# The GASTON autoencoder (next step) will:
#   encoder:  (x_i, y_i)  ->  scalar isodepth d_i
#   decoder:  d_i          ->  A_hat_i in R^14
#   loss:     ||A_i - A_hat_i||^2
#
# For C-GASTON (our project), A is also used to build the molecular
# branch embedding: z_m = W_m * [d_i ; A_hat_i]  (dim = 1+14 = 15 -> 128)
# which is then aligned with the vision branch via InfoNCE contrastive loss.

print("=" * 60)
print("GLM-PCA complete.  Outputs saved:")
print("=" * 60)
for slice_id in slices_to_process:
    out = f'{GLMPCA_OUTPUT_DIR}/{slice_id}'
    print(f"  {slice_id}/")
    print(f"    counts_mat.npy  -> raw X  (N x G)")
    print(f"    coords_mat.npy  -> (x,y)  (N x 2)")
    print(f"    gene_labels.npy -> gene names")
    print(f"    glmpca.npy      -> A      (N x {num_dims})  <-- next step input")

print()
print("Next step: train GASTON neural network (3.Training_Gaston_NN/)")

print()
print("Quick reload example:")
print("""
import numpy as np
d = np.load  # shorthand
sid = '151507'
base = f'{GLMPCA_OUTPUT_DIR}/{sid}'
A          = np.load(f'{base}/glmpca.npy')               # (N, 14)
coords_mat = np.load(f'{base}/coords_mat.npy')           # (N, 2)
""")